In [ ]:
import json
import os
import sys
from glob import glob
from io import StringIO

import networkx as nx
import pandas as pd

os.chdir(r"E:\Files\llm4tg")
sys.path.append(".")


from aux_querygpt import aquery_once, get_llm_model, token_count


tokens = pd.read_csv("tokens.csv")

small_graphs = tokens[tokens[".gexf"] < 125000]
small_graphs_addr = small_graphs["addr"].tolist()
small_graphs_addr

basd8_root = r"E:\Files\llm4tg\basd-8"

subgraphs = glob(basd8_root + r"\**\*.graphml", recursive=True)


def extract_addr(pth):
    return os.path.splitext(os.path.basename(pth))[0]


applicable_graphs = [x for x in subgraphs if extract_addr(x) in small_graphs_addr]
applicable_graphs
applicable_graphs2 = []
for subgraph in applicable_graphs:
    addr = extract_addr(subgraph)
    G: nx.DiGraph = nx.read_graphml(subgraph)
    nodes = G.number_of_nodes()
    if nodes > 10:
        applicable_graphs2.append(subgraph)
applicable_graphs = applicable_graphs2
applicable_graphs

import asyncio
import random
from collections import defaultdict

from tqdm.auto import tqdm


async def main():
    results = {}
    for subgraph in tqdm(applicable_graphs):
        addr, _type = os.path.splitext(os.path.basename(subgraph))
        tqdm.write(f"{addr=}")
        G: nx.DiGraph = nx.read_graphml(subgraph)
        nodes = list(G.nodes)
        depth = nx.single_source_shortest_path_length(
            G.to_undirected(as_view=True), source="n0"
        )
        sorted_depth = {k: depth[k] for k in nodes}
        rev_dict = defaultdict(list)
        [rev_dict[v].append(k) for k, v in sorted_depth.items()]
        selpoints = set(random.choice(l) for l in rev_dict.values())
        while len(selpoints) < 10:
            selpoints.add(random.choice(nodes))
        with open(f"repr_llm4tg/{addr}.llm4tg") as f:
            graph_repr_llm4tg = f.read()
        graph_repr_gexf = "\n".join(nx.generate_gexf(G))
        graph_repr_gml = "\n".join(nx.generate_gml(G))
        with open(subgraph, "r") as f:
            graph_repr_graphml = f.read()
        # tqdm.write(f"{token_count(graph_repr_llm4tg)=}")
        selpoints = sorted(selpoints)
        node_info = {
            n: {
                "in_degree": G.in_degree(n),
                "out_degree": G.out_degree(n),
                "in_value": sum([e["value"] for (u, v, e) in G.in_edges(n, data=True)]),
                "out_value": sum(
                    [e["value"] for (u, v, e) in G.out_edges(n, data=True)]
                ),
            }
            for n in nodes
        }
        node_info = {
            n: {
                **v,
                "diff_degree": abs(v["in_degree"] - v["out_degree"]),
                "diff_value": abs(v["in_value"] - v["out_value"]),
            }
            for n, v in node_info.items()
        }

        def make_questions(n):
            if depth[n] & 1:  # transaction
                in_nodes = [f for f, _ in G.in_edges(n)]
                out_nodes = [t for _, t in G.out_edges(n)]
                should_exist = (ord(n[-1]) - ord("0")) & 1
                conn = set(in_nodes + out_nodes)
                rest = set(nodes) - conn
                chknode = random.choice(list(conn if should_exist else rest))
                return (
                    f"node {n}: answer Input/Output Degree and Amount. And check if {n} is connected with {chknode}, answer in special_info field by true/false.",
                    (chknode, should_exist),
                )
            else:  # address
                times = [
                    x for _, _, x in G.to_undirected(as_view=True).edges(n, data="time")
                ]
                return (
                    f"node {n}: answer Input/Output Degree and Amount. And find out Time range field, answer in special_info.",
                    max(times) - min(times),
                )

        questions = [make_questions(n) for n in selpoints]
        ground_truth = {
            "global": {
                f"max_{key}_node": max(node_info, key=lambda x: node_info[x][key])
                for key in [
                    "in_degree",
                    "out_degree",
                    "in_value",
                    "out_value",
                    "diff_degree",
                    "diff_value",
                ]
            },
            "nodes": {
                n: {
                    "in_degree": node_info[n]["in_degree"],
                    "out_degree": node_info[n]["out_degree"],
                    "in_value": node_info[n]["in_value"],
                    "out_value": node_info[n]["out_value"],
                    "special_info": questions[i][1],
                }
                for i, n in enumerate(selpoints)
            },
        }

        questions_prompt = "\n".join([q[0] for q in questions])
        gpt4o = get_llm_model("gpt-4o")
        task_llm4tg = aquery_once(
            gpt4o, "query_basic", nodes=questions_prompt, graph=graph_repr_llm4tg
        )
        task_gexf = aquery_once(
            gpt4o, "query_basic", nodes=questions_prompt, graph=graph_repr_gexf
        )
        task_gml = aquery_once(
            gpt4o, "query_basic", nodes=questions_prompt, graph=graph_repr_gml
        )
        task_graphml = aquery_once(
            gpt4o, "query_basic", nodes=questions_prompt, graph=graph_repr_graphml
        )
        result_llm4tg, result_gexf, result_gml, result_graphml = await asyncio.gather(
            task_llm4tg, task_gexf, task_gml, task_graphml
        )
        results[addr] = {
            "selected_nodes": selpoints,
            "ground_truth": ground_truth,
            "llm4tg": result_llm4tg,
            "gexf": result_gexf,
            "gml": result_gml,
            "graphml": result_graphml,
        }
        with open(f"lv9_www/exexp2_results.json", "w") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)


await main()

In [1]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://ai-yuexinxiang9466ai381972092863.services.ai.azure.com/models"
model_name = "Llama-3.3-70B-Instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(
        "49iUcYHVmO7eS7e0x1yTzaNqKPWWSmk2QheorOD9sr0msPiDeNt5JQQJ99BDACHYHv6XJ3w3AAAAACOGHE7N"
    ),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="I am going to Paris, what should I see?"),
    ],
    max_tokens=2048,
    temperature=0.8,
    top_p=0.1,
    presence_penalty=0.0,
    frequency_penalty=0.0,
    model=model_name,
)

print(response.choices[0].message.content)

Paris, the City of Light. You're in for a treat. There are countless iconic landmarks, museums, and experiences to explore in Paris. Here are some top recommendations:

**Must-see attractions:**

1. **The Eiffel Tower** (La Tour Eiffel): The iron lady of Paris, offering breathtaking views of the city.
2. **The Louvre Museum** (Musée du Louvre): Home to the Mona Lisa, Venus de Milo, and an impressive collection of art and artifacts.
3. **Notre-Dame Cathedral** (Cathédrale Notre-Dame de Paris): A beautiful, historic church that was built in the 12th century (note: it's currently under renovation after the 2019 fire).
4. **Arc de Triomphe**: A monumental arch honoring the soldiers who fought and died for France, with stunning views of the city from the top.
5. **Champs-Élysées**: A famous avenue lined with cafes, shops, and theaters, perfect for a leisurely stroll.

**Other popular attractions:**

1. **Montmartre**: A charming, bohemian neighborhood with narrow streets, artist studios, an

In [ ]:
import getpass
import os

from dotenv import load_dotenv
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
load_dotenv(override=True)
print(f"{os.getenv('AZURE_INFERENCE_ENDPOINT')=}")
print(f"{os.getenv('AZURE_INFERENCE_CREDENTIAL')=}")

model = AzureAIChatCompletionsModel(
    model_name="llama-3.3-70b-instruct",
)
model.invoke("Hello, world!")

ImportError: cannot import name 'ChatCompletionsResponseFormatJSON' from 'azure.ai.inference.models' (d:\Applications\Python\lib\site-packages\azure\ai\inference\models\__init__.py)